In [1]:
import moviepy
from moviepy.video.VideoClip import ImageClip
from moviepy.editor import VideoFileClip, concatenate_videoclips, ipython_display
from PIL import Image, ImageChops
import numpy as np
from os.path import join as opj
from collections import OrderedDict
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
from movie_tools import mix_frames, scroll_im_frames, supplement_ims, add_scroll_to_text, color_mix_frames, g_to_mono, get_sub_images
from matplotlib import animation
from matplotlib import pyplot as plt
from IPython.display import HTML
from PIL import ImageOps, ImageChops, ImageEnhance
import skimage.measure as measure
from skimage import io, color
import skimage.morphology as morph
from skimage.filters import threshold_otsu
from moviepy.video import VideoClip
from moviepy.editor import AudioFileClip, concatenate_audioclips, vfx, CompositeAudioClip, concatenate_videoclips


In [2]:
def get_animation(frames, interval=100, downsample=None):
    fig,ax=plt.subplots()
    display_im=ax.imshow(frames[0])
    ax.axis("off")
    plt.close()
    
    def init():
        display_im.set_data(frames[0])
    
    def animate(i):
        if downsample:
            im = frames[i].copy()
            new_size = (int(float(im.size[0])/downsample+0.5),
                        int(float(im.size[1])/downsample+0.5)
                       )
            if i==0:
                print("downsampling from %s to %s"%(str(im.size), str(new_size)))
            im=im.resize(new_size)
        else:
            im = frames[i]
        display_im.set_data(im)
    
    return animation.FuncAnimation(fig, animate, init_func=init, frames=len(frames), interval=interval)

def get_sub_image(frac_left, frac_bottom, size_frac_y, size_out, im, verbose=False):
    if isinstance(im, str):
        im = Image.open(im)
    orig_size = im.size
    if verbose:
        print("orig_size:",orig_size)
    left = int(frac_left*orig_size[0]+0.5)
    bottom = int(frac_bottom*orig_size[1]+0.5)
    dy = size_frac_y*orig_size[1]
    top = bottom + int(dy+0.5)
    dx = dy * float(size_out[0])/size_out[1]
    right = left + int(dx+0.5)

    box = [left,bottom,right,top]
    if verbose:
        print("box:",box)
    im_cropped = im.crop(tuple(box))
    
    return im_cropped.resize(size_out,resample=Image.LANCZOS)
    
#get sub-images
def get_sub_images(frac_left, frac_bottom, size_frac_y, size_out, ims, verbose=False, convert=None):
    ims_out = []
    for im in ims:
        if isinstance(im, str):
            im = Image.open(im)
            if convert:
                im = im.convert("L")
        orig_size = im.size
        if verbose:
            print("orig_size:",orig_size)
        left = int(frac_left*orig_size[0]+0.5)
        bottom = int(frac_bottom*orig_size[1]+0.5)
        dy = size_frac_y*orig_size[1]
        top = bottom + int(dy+0.5)
        dx = dy * float(size_out[0])/size_out[1]
        right = left + int(dx+0.5)

        box = [left,bottom,right,top]
        if verbose:
            print("box:",box)
        im_cropped = im.crop(tuple(box))
        ims_out.append(
            im_cropped.resize(size_out,resample=Image.LANCZOS)
        )
    return ims_out

def g_to_mono_grey(im):
    return g_to_mono(im, (0,0,0))

def flicker_marks(im, num_frames, thresh_mult=1.7, alpha_min=10, add_alpha=False):
    thresh = threshold_otsu(np.asarray(im))
    im_grey = (im.copy()).convert("L")
    im_grey_inverted = ImageOps.invert(im_grey.copy())
    im_thresh = (np.asarray(im_grey_inverted) > thresh*thresh_mult)
    Strel = morph.disk(2)
    im_thresh = morph.dilation(im_thresh,Strel)

    label_img = measure.label(im_thresh)
    print("label_img.shape:", label_img.shape)
    #now we've got the labels, we just want to turn on and off
    #marks at random
    num_labels = label_img.max()
    print("num_labels:",num_labels)
    labels = np.arange(1, num_labels+1)
    frames_out = []
    for i in range(num_frames):
        alpha_vals = (255*np.random.random(len(labels))).astype(int)
        im_new = im_grey.copy()
        #Might be easiest to do this with alpha channel
        alpha = 10*np.ones(im_grey.size).astype(np.uint8).T
        for label,alpha_val in zip(labels, alpha_vals):
            alpha[np.where(label_img==label)] = alpha_val
        alpha_im = Image.fromarray(alpha)
        if add_alpha:
            im_new.putalpha(alpha_im)
        else:
            alpha_frac = alpha.astype(float)/255
            im_new_array = np.round( (1-alpha_frac)*255 + alpha_frac*np.array(im_grey.copy()) ).astype(np.uint8)
            im_new = Image.fromarray(im_new_array)
            
        frames_out.append(im_new)
        
    return frames_out, label_img

def reverse_loop_frames(frames, n):
    rev_frames = frames[::-1]
    frames_out=[]
    for i in range(n):
        frames_out += frames
        frames_out += rev_frames[1:-1]
    return frames_out


def lighten_im(im, lighten_fac):
    im_array = np.array(im)
    if im.mode=="RGB":
        im_array_out = np.zeros_like(im_array)
        for i,channel in enumerate(im_array):
            im_array_out[i] = np.round((1-lighten_fac)*255 + lighten_fac*im_array[i]).astype(np.uint8)
        return Image.fromarray(im_array_out)
    elif im.mode=="L":
        im_array_out = np.round((1-lighten_fac)*255 + lighten_fac*im_array).astype(np.uint8)
        return Image.fromarray(im_array_out)

def darken_im(im, darken_fac):
    im_array = np.array(im)
    if im.mode=="RGB":
        im_array_out = np.zeros_like(im_array)
        for i,channel in enumerate(im_array):
            im_array_out[i] = np.round((1-lighten_fac)*0 + darken_fac*im_array[i]).astype(np.uint8)
        return Image.fromarray(im_array_out)
    elif im.mode=="L":
        im_array_out = np.round((1-lighten_fac)*0 + darken_fac*im_array).astype(np.uint8)
        return Image.fromarray(im_array_out)

def come_in_and_fade_out(im, num_frames):
    frames = []
    lighten_facs = np.linspace(1., 0., num_frames)
    print("lighten_facs:",lighten_facs)
    return [lighten_im(im, l) for l in lighten_facs]

def im_to_frame_func(downsample):
    def im_to_frame(frame):
        im = frame.copy()
        if downsample:
            new_size = (int(float(im.size[0])/downsample+0.5),
                        int(float(im.size[1])/downsample+0.5)
                       )
        return np.asarray(im)
    return im_to_frame



def supmult(ims):
    return supplement_ims(ims, combine_func=ImageChops.multiply) 
    
def supblend(ims):
    return supplement_ims(ims, combine_func=ImageChops.blend, combine_func_args={"alpha":0.5}) 
    

In [3]:
im0_g = Image.open(
    "carb_head_0.jpg"
).convert("L")
orig_size=im0_g.size
scale=0.4
if scale is None:
    size = im0_g.size
else:
    size = (int(im0_g.size[0]*scale), int(im0_g.size[1]*scale))
im0_g = im0_g.resize(size)

In [4]:
carb_head_ims = [(Image.open(
    "carb_head_%d.jpg"%i).convert("L")).resize(size) for i in range(5)]
#increase contrast
carb_head_ims = [ImageOps.autocontrast(im, cutoff=(2.,2.)) for im in carb_head_ims ]
carb_head_ims = carb_head_ims + carb_head_ims[::-1][1:-1]

print("len(carb_head_ims)",len(carb_head_ims))

new_carb_head_ims = [(Image.open(
    "new_carb_head_%d.jpg"%i).convert("L")).resize(size) for i in range(3)]
new_carb_head_ims += [new_carb_head_ims[1],new_carb_head_ims[0]]
#increase contrast
new_carb_head_ims = [ImageOps.autocontrast(im, cutoff=(2.,2.)) for im in new_carb_head_ims ]
new_carb_head_ims = new_carb_head_ims + new_carb_head_ims[::-1][1:-1]
print("len(new_carb_head_ims)",len(new_carb_head_ims))

len(carb_head_ims) 8
len(new_carb_head_ims) 8


In [5]:
zoom_new_carb_head_ims = get_sub_images(0.1, 0.3, 0.4, size, ["new_carb_head_%d.jpg"%i for i in [0,1,2]])
zoom_new_carb_head_ims += [zoom_new_carb_head_ims[1],zoom_new_carb_head_ims[0]]
zoom_new_carb_head_ims = zoom_new_carb_head_ims+zoom_new_carb_head_ims[::-1][1:-1]
print(len(zoom_new_carb_head_ims))


8


In [6]:
soft_head_ims = [(Image.open(
    "soft_head_%d.jpg"%i).convert("L")).resize(size) for i in range(5)]
soft_head_ims = [ImageOps.autocontrast(im, cutoff=(10.,5.)) for im in soft_head_ims ]
soft_head_ims = soft_head_ims + soft_head_ims[::-1][1:-1]
soft_head_ims_brown = [g_to_mono(im, rgb=(33, 20, 5), gamma=1.) for im in soft_head_ims]
soft_head_ims_red = [g_to_mono(im, rgb=(181, 100, 60), gamma=1.) for im in soft_head_ims]

In [7]:
text_ims = [(Image.open(
    "text/text%d.jpg"%i).convert("L")).resize(size) for i in range(1,16)]
text_ims += [(Image.open(
    "text/text%d.jpg"%i).convert("L")).resize(size) for i in range(17,26)]

## Audio

In [8]:
rain_audio_clip = AudioFileClip("rain.m4a").subclip(3.,10.)
guitar_audio_clip1 = AudioFileClip("Woody Guthrie- This Land Is Your Land.mp3").subclip(13.4,13.55)
guitar_audio_clip1 = guitar_audio_clip1.fx(vfx.speedx, 0.5)

guitar_audio_clip1_backwards = guitar_audio_clip1.fl_time(lambda t: guitar_audio_clip1.duration-t)
guitar_audio_clip1_backwards.duration = guitar_audio_clip1.duration

guitar_audio_clip1_wbackwards = concatenate_audioclips([guitar_audio_clip1,guitar_audio_clip1_backwards]*5)

guitar_audio_clip1_wbackwards = guitar_audio_clip1_wbackwards.audio_fadeout(guitar_audio_clip1_wbackwards.duration)

ipython_display(guitar_audio_clip1_wbackwards, maxduration=200.)  

MoviePy - Writing audio in __temp__.mp3


MoviePy - Done.


#Let's start putting this shit together
- Start with flickering positive marks for e.g. 5 seconds
- Then, accompanied by a resonant guitar sound, the line comes in

In [9]:
fps=20
downsample=4

In [10]:
frames1_orig = []
fam_inds_1_orig = []

def supplement_fam_inds(fam_inds):
    n_ims = len(fam_inds)
    new_fam_inds = [fam_inds[0]]
    for i in range(0, n_ims-1):
        new_fam_inds.append(fam_inds[i])
        new_fam_inds.append(fam_inds[i+1])
    return new_fam_inds

print(len(zoom_new_carb_head_ims))

for i,zoom_new_carb_head_im in enumerate(zoom_new_carb_head_ims):
    f, _ = flicker_marks(zoom_new_carb_head_im, 10, thresh_mult=1.2)
    #turn green
    f = [g_to_mono(im, rgb=(19, 99, 62), gamma=1.) for im in f]
    frames1_orig+=f
    fam_inds_1_orig+=[i]*len(f)

frames1 = supblend(supplement_ims(frames1_orig))
fam_inds_1 = supplement_fam_inds(supplement_fam_inds(fam_inds_1_orig))
    
print(len(frames1))
print(len(fam_inds_1))
#fam_inds_1 = 

8
label_img.shape: (1392, 1161)
num_labels: 208
label_img.shape: (1392, 1161)
num_labels: 181
label_img.shape: (1392, 1161)
num_labels: 152
label_img.shape: (1392, 1161)
num_labels: 181
label_img.shape: (1392, 1161)
num_labels: 208
label_img.shape: (1392, 1161)
num_labels: 181
label_img.shape: (1392, 1161)
num_labels: 152
label_img.shape: (1392, 1161)
num_labels: 181
317
317


In [11]:
v1 = VideoClip.DataVideoClip(frames1, im_to_frame_func(downsample), fps=fps)

audio1 = concatenate_audioclips([rain_audio_clip] * int(v1.duration/rain_audio_clip.duration+1) )
audio1 = audio1.set_duration(v1.duration)

print(v1.duration)
print(audio1.duration)

v1 = v1.set_audio(audio1) 
#v2.write_videofile("test.mp4",codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False
#)

15.85
15.85


In [12]:
l = concatenate_audioclips([rain_audio_clip] * 10)
ipython_display(l, maxduration=200.)  

MoviePy - Writing audio in __temp__.mp3


MoviePy - Done.


In [13]:
guitar_audio_clip2 = AudioFileClip("Woody Guthrie- This Land Is Your Land.mp3").subclip(5.,7.5)

guitar_audio_clip2_backwards = guitar_audio_clip2.fl_time(lambda t: guitar_audio_clip2.duration-t)
guitar_audio_clip2_backwards.duration = guitar_audio_clip2.duration
print(guitar_audio_clip2_backwards.start)

guitar_audio_clip2_wbackwards = concatenate_audioclips([guitar_audio_clip2,guitar_audio_clip2_backwards])

guitar_audio_clip2_wbackwards = guitar_audio_clip2_wbackwards.fx(vfx.speedx, 0.5)
###guitar_audio_clip2_backwards.duration = guitar_audio_clip2.duration
ipython_display(guitar_audio_clip2_wbackwards, maxduration=200.)  

0
MoviePy - Writing audio in __temp__.mp3


MoviePy - Done.


In [14]:
guitar_audio_clip3 = AudioFileClip("Woody Guthrie- This Land Is Your Land.mp3").subclip(9.8,12)


guitar_audio_clip3_backwards = guitar_audio_clip3.fl_time(lambda t: guitar_audio_clip3.duration-t)
guitar_audio_clip3_backwards.duration = guitar_audio_clip3.duration

guitar_audio_clip3_wbackwards = concatenate_audioclips([guitar_audio_clip3,guitar_audio_clip3_backwards])
guitar_audio_clip3_wbackwards = guitar_audio_clip3_wbackwards.fx(vfx.speedx, 0.5)

#guitar_audio_clip3 = concatenate_audioclips([guitar_audio_clip2_wbackwards,guitar_audio_clip3])

#ipython_display(guitar_audio_clip3_wbackwards, maxduration=200.)  

In [15]:
a = concatenate_audioclips([guitar_audio_clip2_wbackwards, guitar_audio_clip3_wbackwards]*3)

ipython_display(a,maxduration=200)

MoviePy - Writing audio in __temp__.mp3


MoviePy - Done.


In [16]:
frames2_nozoom = []
fam_inds_2 = []

flicker_marks_dict = {}

for i,new_carb_head_im in enumerate(new_carb_head_ims):
    f, _ = flicker_marks(new_carb_head_im, 10, thresh_mult=1.2)
    #turn green
    f = [g_to_mono(im, rgb=(50, 88, 148), gamma=1.) for im in f]
    flicker_marks_dict[i] = f
    frames2_nozoom+=f
    fam_inds_2+=[i]*len(f)

frames2_nozoom = supblend(supplement_ims(frames2_nozoom))
fam_inds_2 = supplement_fam_inds(supplement_fam_inds(fam_inds_2))

#add fading original marks
fade_facs = np.linspace(1.,0.,len(frames1))
frames2 = [ImageChops.multiply(f2, 
                               lighten_im(f1,fade_fac)) for (f2,f1,fade_fac) in zip(frames2_nozoom,frames1,fade_facs)]

label_img.shape: (1392, 1161)
num_labels: 344
label_img.shape: (1392, 1161)
num_labels: 351
label_img.shape: (1392, 1161)
num_labels: 331
label_img.shape: (1392, 1161)
num_labels: 351
label_img.shape: (1392, 1161)
num_labels: 344
label_img.shape: (1392, 1161)
num_labels: 351
label_img.shape: (1392, 1161)
num_labels: 331
label_img.shape: (1392, 1161)
num_labels: 351


## %matplotlib inline
HTML(get_animation(frames2_wzoom, interval=100, downsample=4).to_jshtml())

In [17]:
v2 = VideoClip.DataVideoClip(frames2, im_to_frame_func(downsample), fps=fps)

audio2 = concatenate_audioclips([rain_audio_clip] * int(v2.duration/rain_audio_clip.duration+1) )
audio2 = audio2.set_duration(v2.duration)
print(v2.duration)
print(audio2.duration)


v2 = v2.set_audio(audio2) 
#v2.write_videofile("test.mp4",codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False
#)

15.85
15.85


In [18]:
#ipython_display(v2)

In [19]:
#Bring the line but just a random one
#We're going to loop through the head frames 
#a few times, activating the line image at random
#then fading it out 
num_loops = 1
line_prob = 0.3

dt = 1./fps

duration3 = num_loops * len(frames2) * dt
print(duration3)
audio3 = concatenate_audioclips([rain_audio_clip] * int(duration3/rain_audio_clip.duration+1))
audio3 = audio3.set_duration(duration3)
print(audio3.duration)


frames3 = []
add_line=False
lighten_fac=1.
i_frame=0
for i in range(num_loops):
    im_fam_current = -1
    for i2,f2 in enumerate(frames2):
        f = f2.copy()
        im_fam = fam_inds_2[i2]
        
        if im_fam != im_fam_current:
            if (i==0) and (i2==0):
                add_line = True
            else:
                add_line = np.random.random()<line_prob
            if add_line:
                if im_fam==0:
                    audio3 = CompositeAudioClip([audio3, guitar_audio_clip1_wbackwards.set_start(i_frame*dt)])
                lighten_fac = 1.
        im_fam_current=im_fam
        #print(im_fam,lighten_fac,add_line)
        if add_line:
            soft_head_im = lighten_im(soft_head_ims_brown[im_fam], lighten_fac)
            lighten_fac = max(0., lighten_fac-0.1)
            f = ImageChops.multiply(f, soft_head_im)
        frames3.append( f )  
        i_frame += 1


15.850000000000001
15.850000000000001


In [20]:
#add text
def add_text(frames, text_ims, appear_prob = 0.02,
             advance_prob = 0.5, advance_during_prob=None):
    
    text_ind = 0
    text_is_present = False
    frames_final = []
    lighten_facs = np.concatenate([np.linspace(0.,1.,5),np.linspace(1.,0.,80)])
    blend_facs = np.concatenate([np.zeros(5), np.linspace(1.,0.,10), np.zeros(70)])
    lighten_ind = 0
    can_advance_during = True

    for f_in in frames:
        f_grey = f_in.convert("L")
        alpha = (1-np.array(f_grey).astype(float)/255)**0.5
        #
        text_array = np.round( (1-alpha)*255 + alpha*np.array(text_ims[text_ind].copy()) ).astype(np.uint8)
        text_im = Image.fromarray(text_array)
        text_im = g_to_mono(text_im, (0.,0.,0.))
    
        if text_is_present:
            #print("lighten_ind:",lighten_ind)
            if (advance_during_prob is not None) and can_advance_during:
                if np.random.random()<advance_during_prob:
                    can_advance_during = False
                    if text_ind == len(text_ims)-1:
                        text_ind -= 2
                    else:
                        text_ind += 1
                
            if lighten_ind==0:
                print("text_ind:",text_ind)
            f = ImageChops.multiply(f_in, lighten_im(text_im, lighten_facs[lighten_ind]))
            f = ImageChops.blend(f, g_to_mono_grey(text_ims[text_ind]), alpha=blend_facs[lighten_ind])
            frames_final.append(
                f
            )
            if lighten_ind == len(lighten_facs)-1:
                text_is_present = False
                can_advance_during = True
                if text_ind == len(text_ims)-1:
                    text_ind = 0
                else:
                    if np.random.random()<advance_prob:
                        text_ind += 1
                lighten_ind = 0
            else:
                lighten_ind += 1
    
        else:
            frames_final.append(f_in)
            text_is_present = np.random.random()<appear_prob

    return frames_final
            


In [21]:
v3 = VideoClip.DataVideoClip(frames3, im_to_frame_func(downsample), fps=fps)
print(audio3.duration)
print(v3.duration)
audio3 = audio3.set_duration(v3.duration)
#v3.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=True)
#
#print(len(frames3) * 1./fps)
#print(v3.duration)
#print(audio3.duration)
#v3 = v3.set_audio(audio3) 

15.850000000000001
15.85


In [22]:
frames_2to3_withtext = add_text(frames2+frames3, text_ims)
audio_1to3 = concatenate_audioclips([audio1,audio2,audio3])
v = VideoClip.DataVideoClip(frames1+frames_2to3_withtext, im_to_frame_func(8), fps=fps)
v = v.set_audio(audio_1to3)

#v.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=True)

text_ind: 0
text_ind: 1
text_ind: 2
text_ind: 3
text_ind: 4


In [23]:
#Now the same but also bring the red line in at random
#Bring the line but just a random one
#We're going to loop through the head frames 
#a few times, activating the line image at random
#then fading it out 
num_loops = 1
line_prob = 0.8

duration4 = num_loops * len(frames2) * 1./fps
audio4 = concatenate_audioclips([rain_audio_clip] * int(duration4/rain_audio_clip.duration+1))
audio4 = audio4.set_duration(duration4)

frames4 = []
add_line=False
add_line_red=False
lighten_fac=1.
lighten_fac_red=1.
i_frame=0
for i in range(num_loops):
    im_fam_current = -1
    for i2,f2 in enumerate(frames2):
        f = f2.copy()
        im_fam = fam_inds_2[i2]
        
        if im_fam != im_fam_current:
            #if (i==0) and (i2==0):
            #else:
            #    add_line = True
            add_line = np.random.random()<line_prob
            add_line_red = np.random.random()<line_prob
            print("new_fam:")
            print("add_line_red:",add_line_red)
            if add_line:
                if im_fam==0:
                    audio4 = CompositeAudioClip([audio4, guitar_audio_clip1_wbackwards.set_start(i_frame*dt)])
            lighten_fac = 1.
            lighten_fac_red = 1.

        im_fam_current=im_fam
        #print(im_fam,lighten_fac,add_line)
        if add_line:
            soft_head_im = lighten_im(soft_head_ims_brown[im_fam], lighten_fac)
            lighten_fac = max(0., lighten_fac-0.1)
            f = ImageChops.multiply(f, soft_head_im)
        if add_line_red:
            im_fam_red = im_fam+1 if im_fam<len(soft_head_ims_brown)-1 else 0
            soft_head_im_red = lighten_im(soft_head_ims_red[im_fam], lighten_fac_red)
            lighten_fac_red = max(0., lighten_fac_red-0.1)
            f = ImageChops.multiply(f, soft_head_im_red)
        frames4.append( f )  
        i_frame += 1

new_fam:
add_line_red: True
new_fam:
add_line_red: True
new_fam:
add_line_red: True
new_fam:
add_line_red: True
new_fam:
add_line_red: True
new_fam:
add_line_red: True
new_fam:
add_line_red: True
new_fam:
add_line_red: True


In [24]:
v4 = VideoClip.DataVideoClip(frames4, im_to_frame_func(downsample), fps=fps)
print(v4.duration)
print(audio4.duration)
v4 = v4.set_audio(audio4) 

15.85
15.85


In [25]:
frames_2to4_withtext = add_text(frames2+frames3+frames4, text_ims, advance_prob = 1)
audio_1to4 = concatenate_audioclips([audio1,audio2,audio3,audio4])
print(audio_1to4.duration)
v_1to4 = VideoClip.DataVideoClip(frames1+frames_2to4_withtext, im_to_frame_func(8), fps=fps)
print(v_1to4.duration)
v_1to4 = v_1to4.set_audio(audio_1to4)

#v_1to4.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=True)

text_ind: 0
text_ind: 1
text_ind: 2
text_ind: 3
text_ind: 4
63.4
63.4


In [26]:
#Now the same but also bring the red line in at random
#Bring the line but just a random one
#We're going to loop through the head frames 
#a few times, activating the line image at random
#then fading it out 
num_loops = 1
line_prob = 0.8
line_prob_red = 0.5

duration5 = num_loops * len(frames2) * 1./fps
audio5 = concatenate_audioclips([rain_audio_clip] * int(duration5/rain_audio_clip.duration+1))
audio5 = audio5.set_duration(duration5)

frames5 = []
add_line=False
add_line_red=False
lighten_fac=1.
lighten_fac_red=1.
i_frame=0
for i in range(num_loops):
    im_fam_current = -1
    for i2,f2 in enumerate(frames2):
        f = f2.copy()
        im_fam = fam_inds_2[i2]
        
        if im_fam != im_fam_current:
            #if (i==0) and (i2==0):
            #else:
            #    add_line = True
            add_line = np.random.random()<line_prob
            add_line_red = np.random.random()<line_prob_red
            print("new_fam:")
            print("add_line_red:",add_line_red)
            if add_line:
                if im_fam==0:
                    audio5 = CompositeAudioClip([audio5, guitar_audio_clip1_wbackwards.set_start(i_frame*dt)])
            lighten_fac = 1.
            lighten_fac_red = 1.

        im_fam_current=im_fam
        #print(im_fam,lighten_fac,add_line)
        if add_line:
            soft_head_im = lighten_im(soft_head_ims_brown[im_fam], lighten_fac)
            lighten_fac = max(0., lighten_fac-0.1)
            f = ImageChops.multiply(f, soft_head_im)
        if add_line_red:
            im_fam_red = im_fam+1 if im_fam<len(soft_head_ims_brown)-1 else 0
            soft_head_im_red = lighten_im(soft_head_ims_red[im_fam], lighten_fac_red)
            lighten_fac_red = max(0., lighten_fac_red-0.1)
            f = ImageChops.multiply(f, soft_head_im_red)
        frames5.append( f )  
        i_frame += 1

new_fam:
add_line_red: True
new_fam:
add_line_red: True
new_fam:
add_line_red: False
new_fam:
add_line_red: False
new_fam:
add_line_red: True
new_fam:
add_line_red: True
new_fam:
add_line_red: False
new_fam:
add_line_red: True


In [27]:
v5 = VideoClip.DataVideoClip(frames5, im_to_frame_func(downsample), fps=fps)
print(v5.duration)
print(audio5.duration)
v5 = v5.set_audio(audio5) 


#v5.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)

15.85
15.85


In [28]:
#Now the same but also bring the red line in at random
#Bring the line but just a random one
#We're going to loop through the head frames 
#a few times, activating the line image at random
#then fading it out 
num_loops = 1
line_prob = 1.
line_prob_red = 1.

duration6 = num_loops * len(frames2) * 1./fps
audio6 = concatenate_audioclips([rain_audio_clip] * int(duration6/rain_audio_clip.duration+1))
audio6 = audio6.set_duration(duration6)

frames6 = []
add_line=False
add_line_red=False
lighten_fac=1.
lighten_fac_red=1.
dlighten=0.05
i_frame=0
for i in range(num_loops):
    im_fam_current = -1
    for i2,f2 in enumerate(frames2):
        f = f2.copy()
        im_fam = fam_inds_2[i2]
        
        if im_fam != im_fam_current:
            #if (i==0) and (i2==0):
            #else:
            #    add_line = True
            add_line = np.random.random()<line_prob
            add_line_red = np.random.random()<line_prob_red
            if add_line:
                audio6 = CompositeAudioClip([audio6, guitar_audio_clip1_wbackwards.set_start(i_frame*dt)])
            lighten_fac = 1.
            lighten_fac_red = 1.

        im_fam_current=im_fam
        #print(im_fam,lighten_fac,add_line)
        if add_line:
            soft_head_im = lighten_im(soft_head_ims_brown[im_fam], lighten_fac)
            lighten_fac = max(0., lighten_fac-dlighten)
            f = ImageChops.multiply(f, soft_head_im)
        if add_line_red:
            im_fam_red = im_fam+1 if im_fam<len(soft_head_ims_brown)-1 else 0
            soft_head_im_red = lighten_im(soft_head_ims_red[im_fam], lighten_fac_red)
            lighten_fac_red = max(0., lighten_fac_red-dlighten)
            f = ImageChops.multiply(f, soft_head_im_red)
        frames6.append( f )  
        i_frame += 1


#Now bring in the 
#carb head ims
carb_head_ims_green = [g_to_mono(im, rgb=(19, 99, 62), gamma=1.) for im in carb_head_ims]
frames_carb_head = []
for i in range(8):
    inds_use = np.where(np.array(fam_inds_2)==i)[0]
    half = len(inds_use)//2
    lighten_facs = np.zeros_like(inds_use).astype(float)
    lighten_facs[:half] = np.linspace(0.,1.,half)
    lighten_facs[-half:] = np.linspace(1.,0.,half)
    frames_carb_head_i = [lighten_im(carb_head_ims_green[i],l) for l in lighten_facs]
    frames_carb_head += frames_carb_head_i

frames6 = [ImageChops.multiply(f6, f) for (f6,f) in zip(frames6,frames_carb_head)]
print(len(frames6))

317


In [29]:
v6 = VideoClip.DataVideoClip(frames6, im_to_frame_func(downsample), fps=fps)
print(v6.duration)
print(audio6.duration)
v6 = v6.set_audio(audio6) 

#v = concatenate_videoclips([v2,v3,v4,v5,v6])

#v6.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                   remove_temp=False)

15.85
17.000000000000007


In [30]:
frames_2to6_withtext = add_text(frames2+frames3+frames4+frames5+frames6, text_ims[:8], advance_prob=0.7, advance_during_prob=0.02)

audio_1to6 = concatenate_audioclips([audio1,audio2,audio3,audio4,audio5,audio6])
print(audio_1to6.duration)
v_1to6 = VideoClip.DataVideoClip(frames1+frames_2to6_withtext, im_to_frame_func(downsample), fps=fps)
audio_1to6 = audio_1to6.set_duration(v_1to6.duration)
v_1to6 = v_1to6.set_audio(audio_1to6)

v_1to6.write_videofile("test.mp4", codec='libx264', 
                     audio_codec='aac', 
                     temp_audiofile='temp-audio.m4a', 
                     remove_temp=True)

text_ind: 0
text_ind: 2
text_ind: 3
text_ind: 4
text_ind: 6
text_ind: 0
text_ind: 1
text_ind: 2
text_ind: 4
text_ind: 7
text_ind: 0
96.25
Moviepy - Building video test.mp4.
MoviePy - Writing audio in temp-audio.m4a


MoviePy - Done.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4


In [31]:
# Ok we'll call that the intro. 
# Now for when the music comes in I think we want
# to keep the flickers

In [32]:
#v_up_to_6.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)
#

In [33]:
duration7 = 5
im_fam = 0
ind_in_fam = 0
advance_prob = 0.2
reverse_prob = 0.2

print(len(flicker_marks_dict))
frames7 = []
t = 0
while t<duration7:
    f = flicker_marks_dict[im_fam][ind_in_fam]
    f = ImageChops.multiply(f, lighten_im(carb_head_ims_green[im_fam],0.5))
    im_fam_red = im_fam+1 if im_fam<len(soft_head_ims_brown)-1 else 0
    lighten_brown = min(1., 0.5+float(ind_in_fam)*0.25)
    f = ImageChops.multiply(f, lighten_im(soft_head_ims_brown[im_fam], lighten_brown))
    lighten_red = float(ind_in_fam)/len(flicker_marks_dict[im_fam])
    f = ImageChops.multiply(f, lighten_im(soft_head_ims_red[im_fam_red], lighten_red))

    frames7.append(f)
    ind_in_fam = ind_in_fam+1 if ind_in_fam<len(flicker_marks_dict[im_fam])-1 else 0
    if np.random.random()<advance_prob:
        im_fam = im_fam+1 if im_fam<len(soft_head_ims_brown)-1 else 0
    t += 1./fps

frames7 = supblend(supblend(frames7))

8


In [34]:
%matplotlib inline

In [35]:
v7 = VideoClip.DataVideoClip(frames7, im_to_frame_func(downsample), fps=fps)


#v7.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)

In [36]:
# We need to bring the body frames in too. 
# We can use a similar generating function as the last frames, but 
# we need to account for the fact that we don't have corresponding flicker frames 

In [37]:
duration8 = 5
im_fam = 0
ind_in_fam = 0
advance_prob = 0.2
reverse_prob = 0.2

print(len(flicker_marks_dict))
frames8 = []
t = 0

flicker_marks_dict_wbody = dict(flicker_marks_dict)
for i in range(3):
    flicker_marks_dict_wbody[len(flicker_marks_dict)+i] = flicker_marks_dict[i].copy()

#We need to pad out the flicker marks since we only had 8 frames of those, from the 0,1,2,3,4,3,2,1
#sequence of head images 
flicker_marks_dict_wbody = {}
for i in range(5):
    flicker_marks_dict_wbody[i] = flicker_marks_dict[i]
#Now three extra for the body frames 
for i in range(3):
    flicker_marks_dict_wbody[5+i] = flicker_marks_dict[i]
#Now two extra for the body frames in reverse
for i in range(2):
    flicker_marks_dict_wbody[8+i] = flicker_marks_dict[i]
#Now the reverse head ims
for i in range(4):
    flicker_marks_dict_wbody[10+i] = flicker_marks_dict[4+i]

print(flicker_marks_dict_wbody.keys())
    

carb_ims_green = list(carb_head_ims_green)
carb_body_ims = [
    Image.open("carb_body_%d.jpg"%i).convert("L") for i in [1,2,4]]
carb_body_ims = get_sub_images(0.05, 0., 0.9, size, carb_body_ims)
carb_body_ims_green = [g_to_mono(im, rgb=(19, 99, 62), gamma=1.) for im in carb_body_ims]

carb_head_ims_green_orig = carb_head_ims_green[:5]
carb_ims_green = (carb_head_ims_green_orig
                  + carb_body_ims_green 
                  + carb_body_ims_green[::-1][1:-1] 
                  + carb_head_ims_green_orig[::-1][:-1]
                 )

print(len(carb_ims_green))

#carb_ims_green += carb_head_ims_green + carb_body_ims_green

soft_body_ims = [
    Image.open("soft_body_%d.jpg"%i).convert("L") for i in [1,2,4]]
soft_body_ims = get_sub_images(0.05, 0., 0.9, size, soft_body_ims)

soft_head_ims_orig = soft_head_ims[:5]
soft_ims = (soft_head_ims_orig
            + soft_body_ims
            + soft_body_ims[::-1][1:-1]
            + soft_head_ims_orig[::-1][:-1]
           )
print(len(soft_ims))
soft_ims_brown = [g_to_mono(im, rgb=(33, 20, 5), gamma=1.) for im in soft_ims]
soft_ims_red = [g_to_mono(im, rgb=(181, 100, 60), gamma=1.) for im in soft_ims]

#soft_ims_brown 

#soft_ims_red = soft_head_ims_red + soft_body_ims_red
#soft_ims_brown = soft_head_ims_brown + soft_body_ims_brown

while t<duration8:
    f = flicker_marks_dict_wbody[im_fam][ind_in_fam]
    f = ImageChops.multiply(f, lighten_im(carb_ims_green[im_fam],0.5))
    im_fam_red = im_fam+1 if im_fam<len(soft_ims_brown)-1 else 0
    lighten_brown = min(1., 0.5+float(ind_in_fam)*0.25)
    f = ImageChops.multiply(f, lighten_im(soft_ims_brown[im_fam], lighten_brown))
    lighten_red = float(ind_in_fam)/len(flicker_marks_dict_wbody[im_fam])
    f = ImageChops.multiply(f, lighten_im(soft_ims_red[im_fam_red], lighten_red))

    frames8.append(f)
    ind_in_fam = ind_in_fam+1 if ind_in_fam<len(flicker_marks_dict_wbody[im_fam])-1 else 0
    if np.random.random()<advance_prob:
        im_fam = im_fam+1 if im_fam<len(soft_ims_brown)-1 else 0
    t += 1./fps


frames8 = supblend(supblend(frames8))

8
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
13
13


In [38]:
frames8_notext = frames8
frames8 = add_text(frames8_notext, text_ims[9:16], advance_prob=0.5, advance_during_prob=0.1)

text_ind: 0
text_ind: 1


In [39]:
v8 = VideoClip.DataVideoClip(frames8, im_to_frame_func(downsample), fps=fps)

v8.write_videofile("test.mp4", codec='libx264', 
                     audio_codec='aac', 
                     temp_audiofile='temp-audio.m4a', 
                     remove_temp=False)

Moviepy - Building video test.mp4.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4


## let's bring in the raven shall we 

In [40]:
raven_ims = raven_ims=[Image.open("birds/raven%d.jpg"%i).convert("L").resize(size) for i in range(13)]

raven_ims = [ImageOps.autocontrast(im, cutoff=(20.,20.)) for im in raven_ims ]


In [41]:
raven_ims_use = raven_ims
raven_ims_use = raven_ims_use + raven_ims_use[::-1][1:-1]
raven_ims_use = supblend(raven_ims_use)

In [42]:
## This produces a really nice little raven thing, 
## but I don't think 

raven_frames = []

raven_im_ind = 0
advance_prob = 0.3

for i,f8 in enumerate(frames8):
    f8_grey = f8.convert("L")
    alpha = (1-np.array(f8_grey).astype(float)/255)**0.5  
    #could do e.g. 1-0.5*np.array(f8_grey).astype(float)/255 
    #or (1-np.array(f8_grey).astype(float)/255)**0.5  
    #to make raven darker

    raven_array = np.round( (1-alpha)*255 + alpha*np.array(raven_ims_use[raven_im_ind].copy()) ).astype(np.uint8)
    raven_im = Image.fromarray(raven_array)
    raven_im = g_to_mono(raven_im, (0.,0.,0.))

    #print(raven_im_ind, len(raven_ims_use)-1)
    if np.random.random()<advance_prob:
        raven_im_ind = raven_im_ind+1 if raven_im_ind<len(raven_ims_use)-1 else 0
        
    raven_frames.append(raven_im)

In [43]:
#Ok so for the first raven loop,
#just blend into previous frames,
#and set alpha of raven from previous frames
#this is pretty ghosty

In [44]:
frames9 = []

raven_im_ind = 0

raven_im_ind = 0
frames_per_raven = 3 # number of frames per raven image

n_frames_this_raven = 0
num_raven_loops = 1
raven_loops_done = 0

for i,f8 in enumerate(frames8_notext):
    raven_im = raven_ims_use[raven_im_ind]
    thresh = threshold_otsu(np.asarray(raven_im))
    raven_im_inverted = ImageOps.invert(raven_im.copy())
    im_thresh = (np.asarray(raven_im_inverted) > thresh)
    grey_inds = np.where(im_thresh)
    
    #f8_grey_array = np.array(f8.convert("L").convert("RGB"))
    #frame_array[grey_inds] = f8_grey_array[grey_inds]
    #frame_array = np.array(f8.copy())
    #frame = Image.fromarray(frame_array)
    f8_grey = f8.convert("L")
    alpha = (1-np.array(f8_grey).astype(float)/255)**0.5
#
    raven_array = np.round( (1-alpha)*255 + alpha*np.array(raven_ims_use[raven_im_ind].copy()) ).astype(np.uint8)
    raven_im = Image.fromarray(raven_array)
    raven_im = g_to_mono(raven_im, (0.,0.,0.))


    if n_frames_this_raven == frames_per_raven:
        if raven_im_ind<len(raven_ims_use)-1:
            raven_im_ind = raven_im_ind+1 
        else: 
            raven_im_ind = 0
            raven_loops_done += 1
            if raven_loops_done == num_raven_loops:
                break
        n_frames_this_raven = 0
    else:
        n_frames_this_raven += 1

    frames9.append(raven_im)

frames9 = [ImageChops.multiply(f8, f9) for (f9,f8) in zip(frames9,frames8)]
print(len(frames9))

187


In [45]:
v9 = VideoClip.DataVideoClip(frames9, im_to_frame_func(downsample), fps=fps)

In [46]:
#For the next set
#

In [47]:
raven_frames = []

raven_im_ind = 0
frames_per_raven = 3 # number of frames per raven image

alpha_facs = np.linspace(1., 0., len(frames8))

n_frames_this_raven = 0
num_raven_loops = 1
raven_loops_done = 0

for i,f8 in enumerate(frames8_notext):

    raven_im = raven_ims_use[raven_im_ind]
    f8_grey = f8.convert("L")
    alpha = (1-alpha_facs[i]*np.array(f8_grey).astype(float)/255)
#
    raven_array = np.round( (1-alpha)*255 + alpha*np.array(raven_ims_use[raven_im_ind].copy()) ).astype(np.uint8)
    raven_im = Image.fromarray(raven_array)
    raven_im = g_to_mono(raven_im, (0.,0.,0.))

    if n_frames_this_raven == frames_per_raven:
        if raven_im_ind<len(raven_ims_use)-1:
            raven_im_ind = raven_im_ind+1 
        else: 
            raven_im_ind = 0
            raven_loops_done += 1
            if raven_loops_done == num_raven_loops:
                break
        n_frames_this_raven = 0
    else:
        n_frames_this_raven += 1

    
    raven_frames.append(raven_im)

alphas = np.linspace(0.,1.,len(frames9))**0.5
print(len(raven_frames))
print(len(frames9))
frames10 = [ImageChops.blend(f9, r, alpha=a) for (r,f9,a) in zip(raven_frames,frames9,alphas)]

187
187


In [48]:
v10 = VideoClip.DataVideoClip(frames10, im_to_frame_func(downsample), fps=fps)

In [49]:
#We wanted to do some zoooming towards the end of the raven 
#then we're going to stay zoomed in and somehow dark...
raven_zoom_ims = []
raven_file_names = ["birds/raven/raven%d.jpg"%i for i in range(13)]
frac_lefts = np.linspace(0.,0.3,len(raven_file_names)) 
frac_bottoms = np.linspace(0.,0.3, len(raven_file_names))
size_frac_ys = np.linspace(1.,0.3, len(raven_file_names))
for i,f in enumerate(raven_file_names):
    raven_zoom_ims.append(
        get_sub_image(frac_lefts[i], frac_bottoms[i], size_frac_ys[i], size, f).convert("L")
    )

raven_zoomed_ims = []
for i,f in enumerate(raven_file_names):
    raven_zoomed_ims.append(
        get_sub_image(frac_lefts[-1], frac_bottoms[-1], size_frac_ys[-1], size, f).convert("L")
    )

In [50]:
frames11 = []

raven_im_ind = 0
frames_per_raven = 4 # number of frames per raven image

n_frames_this_raven = 1
num_raven_loops = 1
raven_loops_done = 0

raven_ims_use_zoom = supblend(list(raven_zoom_ims)+list(raven_zoomed_ims))
alpha_blends = np.linspace(0.,1.,len(raven_ims_use_zoom)*frames_per_raven)**0.5

print(len(alpha_blends))

for i,f8 in enumerate(frames8_notext):

    raven_im = raven_ims_use_zoom[raven_im_ind]
    thresh = threshold_otsu(np.asarray(raven_im))
    raven_im_inverted = ImageOps.invert(raven_im.copy())
    im_thresh = (np.asarray(raven_im_inverted) > thresh)
    grey_inds = np.where(im_thresh)
    
    #f8_grey_array = np.array(f8.convert("L").convert("RGB"))
    #frame_array[grey_inds] = f8_grey_array[grey_inds]
    #frame_array = np.array(f8.copy())
    #frame = Image.fromarray(frame_array)
    f8_grey = f8.convert("L")
    alpha = (1-np.array(f8_grey).astype(float)/255)
#
    raven_array = np.round( (1-alpha)*255 + alpha*np.array(raven_ims_use_zoom[raven_im_ind].copy()) ).astype(np.uint8)
    raven_im_alpha = Image.fromarray(raven_array)
    raven_im_alpha = g_to_mono(raven_im_alpha, (0.,0.,0.))
    #print(raven_im_alpha, raven_im)
    
    raven_im_use = ImageChops.multiply(raven_im_alpha, 
                                       lighten_im(
                                           g_to_mono(raven_im, (0.,0.,0.)), alpha_blends[i]
                                       )
                                      )

    #print(i, raven_im_ind, n_frames_this_raven)
    
    if n_frames_this_raven == frames_per_raven:
        if raven_im_ind<len(raven_ims_use_zoom)-1:
            raven_im_ind = raven_im_ind+1 
        else: 
            raven_im_ind = 0
            raven_loops_done += 1
            if raven_loops_done == num_raven_loops:
                break
        n_frames_this_raven = 1
    else:
        n_frames_this_raven += 1
    
    frames11.append(raven_im_use)

204


In [51]:
v11 = VideoClip.DataVideoClip(frames11, im_to_frame_func(downsample), fps=fps)

v = concatenate_videoclips([v10,v11])

#v.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)

Now we really want to get these zoom frames darker...
Add in some random frames 

In [52]:
frames12 = []

raven_im_ind = 0
frames_per_raven = 4 # number of frames per raven image

n_frames_this_raven = 1
num_raven_loops = 1
raven_loops_done = 0

raven_ims_use_zoomed = supblend(raven_zoomed_ims)
alpha_blends = np.linspace(0.,1.,len(raven_ims_use_zoomed)*frames_per_raven)**0.5

for i,f8 in enumerate(frames8):

    raven_im = raven_ims_use_zoomed[raven_im_ind]
    thresh = threshold_otsu(np.asarray(raven_im))
    raven_im_inverted = ImageOps.invert(raven_im.copy())
    im_thresh = (np.asarray(raven_im_inverted) > thresh)
    grey_inds = np.where(im_thresh)
    
    f8_grey = f8.convert("L")
    alpha = (1-np.array(f8_grey).astype(float)/255)
#
    raven_array = np.round( (1-alpha)*255 + alpha*np.array(raven_ims_use_zoomed[raven_im_ind].copy()) ).astype(np.uint8)
    raven_im_alpha = Image.fromarray(raven_array)
    raven_im_alpha = g_to_mono(raven_im_alpha, (0.,0.,0.))
    #print(raven_im_alpha, raven_im)
    
    raven_im_use = ImageChops.multiply(raven_im_alpha, 
                                           g_to_mono(raven_im, (0.,0.,0.))
                                      )
    #multiply in some stuff to darken up
    #alpha_blends makes these images progressively darker 
    for j in range(10):
        raven_im_use = ImageChops.multiply(raven_im_use, g_to_mono(lighten_im(frames8[i+j],alpha_blends[i]).convert("L"),(0.,0.,0,)))

    #print(i, raven_im_ind, n_frames_this_raven)
    
    if n_frames_this_raven == frames_per_raven:
        if raven_im_ind<len(raven_ims_use_zoomed)-1:
            raven_im_ind = raven_im_ind+1 
        else: 
            raven_im_ind = 0
            raven_loops_done += 1
            if raven_loops_done == num_raven_loops:
                break
        n_frames_this_raven = 1
    else:
        n_frames_this_raven += 1
    
    frames12.append(raven_im_use)

In [53]:
v12 = VideoClip.DataVideoClip(frames12, im_to_frame_func(downsample), fps=fps)

v = concatenate_videoclips([v10,v11,v12])

#v.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)

In [54]:

v7to12 = concatenate_videoclips([v7,v8,v9,v10,v11,v12])
audio7to12 = concatenate_audioclips([guitar_audio_clip2_wbackwards] * int(v7to12.duration/guitar_audio_clip2_wbackwards.duration+1) )
audio7to12_rain = concatenate_audioclips([rain_audio_clip] * int(v7to12.duration/rain_audio_clip.duration+1))
audio7to12 = CompositeAudioClip([audio7to12, audio7to12_rain])
audio7to12 = audio7to12.set_duration(v7to12.duration)
v7to12 = v7to12.set_audio(audio7to12)

v_upto_12 = concatenate_videoclips([v_1to6, v7to12])

#v_upto_12.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)
#

## Now we're in the dark field 
- Take the aquatint head images and flicker in white marks
- There is quite a lot of playing around possible here.
- Combining with a dark image via hard_light makes for more prominent white marks and dark field - see e.g. aquatint_head_ims_dark


In [55]:
#Same as previous frames, but all dark now (remove lightening alpha_blends)

dark_frames = []
raven_im_ind = 0
frames_per_raven = 4 # number of frames per raven image

n_frames_this_raven = 1
num_raven_loops = 1
raven_loops_done = 0

raven_ims_use_zoomed = supblend(raven_zoomed_ims)

for i,f8 in enumerate(frames8):

    raven_im = raven_ims_use_zoomed[raven_im_ind]
    thresh = threshold_otsu(np.asarray(raven_im))
    raven_im_inverted = ImageOps.invert(raven_im.copy())
    im_thresh = (np.asarray(raven_im_inverted) > thresh)
    grey_inds = np.where(im_thresh)
    
    f8_grey = f8.convert("L")
    alpha = (1-np.array(f8_grey).astype(float)/255)
#
    raven_array = np.round( (1-alpha)*255 + alpha*np.array(raven_ims_use_zoomed[raven_im_ind].copy()) ).astype(np.uint8)
    raven_im_alpha = Image.fromarray(raven_array)
    raven_im_alpha = g_to_mono(raven_im_alpha, (0.,0.,0.))
    #print(raven_im_alpha, raven_im)
    
    raven_im_use = ImageChops.multiply(raven_im_alpha, 
                                           g_to_mono(raven_im, (0.,0.,0.))
                                      )
    #multiply in some stuff to darken up
    for j in range(5):
        raven_im_use = ImageChops.multiply(
            raven_im_use, 
            g_to_mono(
                (frames8[i+j]).convert("L"),
                (0.,0.,0,)
            )
        )

    #print(i, raven_im_ind, n_frames_this_raven)
    
    if n_frames_this_raven == frames_per_raven:
        if raven_im_ind<len(raven_ims_use_zoomed)-1:
            raven_im_ind = raven_im_ind+1 
        else: 
            raven_im_ind = 0
            raven_loops_done += 1
            if raven_loops_done == num_raven_loops:
                break
        n_frames_this_raven = 1
    else:
        n_frames_this_raven += 1
    
    dark_frames.append(raven_im_use)

In [56]:
#add text
def add_text_dark(frames, text_ims, appear_prob = 0.02,
             advance_prob = 0.8, text_bg_ims=raven_ims_use_zoomed*5,
                 advance_during_prob = 0.02, verbose=False):
    
    text_ind = 0
    text_is_present = False
    frames_final = []
    lighten_facs = np.concatenate([np.linspace(0.,1.,5),np.linspace(1.,0.,30)])
    blend_facs = np.concatenate([np.zeros(5), np.linspace(1.,0.,10), np.zeros(20)])
    lighten_ind = 0
    can_advance_during = True
    for f_in in frames:
        frame_grey = f_in.convert("L")
        if text_bg_ims:
            frame_grey = text_bg_ims[lighten_ind].convert("L")
        
    
        if text_is_present:
            if (advance_during_prob is not None) and can_advance_during:
                if np.random.random()<advance_during_prob:
                    can_advance_during = False
                    if text_ind == len(text_ims)-1:
                        text_ind -= 2
                    else:
                        text_ind += 1
            if verbose:
                print("text_ind, lighten_ind:",text_ind,lighten_ind)
            text_im = ImageOps.invert(text_ims[text_ind]).convert("L")
            
            lighten_fac = lighten_facs[lighten_ind]
            alpha_from_text = np.array(text_im).astype(float)/255
            text_array = np.round(alpha_from_text * np.array(ImageOps.invert(frame_grey))).astype(np.uint8)
            im_new = Image.fromarray(text_array)

            im_new = ImageChops.add(f_in, 
                                    darken_im(g_to_mono_grey(im_new), 
                                              lighten_fac))
            #im_new = ImageChops.multiply(f_in, 
            #                        g_to_mono_grey(lighten_im(text_im,blend_facs[lighten_ind]))
            #                            )
                                              
            frames_final.append(
                im_new
            )
            if lighten_ind == len(lighten_facs)-1:
                text_is_present = False
                if text_ind == len(text_ims)-1:
                    text_ind = 0
                else:
                    if np.random.random()<advance_prob:
                        text_ind += 1
                lighten_ind = 0
            else:
                lighten_ind += 1
    
        else:
            frames_final.append(f_in)
            text_is_present = np.random.random()<appear_prob

    return frames_final

In [57]:
frames13 = dark_frames 
v13 = VideoClip.DataVideoClip(dark_frames, im_to_frame_func(downsample), fps=fps)

In [58]:
dark_frames_wtext = add_text_dark(dark_frames, text_ims[16:], appear_prob = 0.1,
             advance_prob = 0.8,
             advance_during_prob = 0.02)
v = VideoClip.DataVideoClip(dark_frames_wtext, im_to_frame_func(8), fps=fps)
#v.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)

In [59]:
aquatint_head_ims = [(Image.open(
    "aquatint_head_%d.jpg"%i).convert("L")).resize(size) for i in range(5)]
#increase contrast
aquatint_head_ims = [ImageOps.autocontrast(im, cutoff=(2.,2.)) for im in aquatint_head_ims ]

aquatint_body_ims = [
    Image.open("aquatint_body_%d.jpg"%i).convert("L") for i in [1,2]]
aquatint_body_ims = get_sub_images(0.05, 0., 0.9, size, aquatint_body_ims)

aquatint_head_ims += aquatint_body_ims

aquatint_head_ims = aquatint_head_ims + aquatint_head_ims[::-1][1:-1]





zoom_aquatint_head_ims = get_sub_images(0.1, 0.3, 0.4, size, ["aquatint_head_%d.jpg"%i for i in range(5)], convert="L")
zoom_aquatint_head_ims = zoom_aquatint_head_ims+zoom_aquatint_head_ims[::-1][1:-1]
print(zoom_aquatint_head_ims)

[<PIL.Image.Image image mode=L size=1161x1392 at 0x13AE13DD0>, <PIL.Image.Image image mode=L size=1161x1392 at 0x13AE13EF0>, <PIL.Image.Image image mode=L size=1161x1392 at 0x13AE07B90>, <PIL.Image.Image image mode=L size=1161x1392 at 0x13AE11760>, <PIL.Image.Image image mode=L size=1161x1392 at 0x13AE2B410>, <PIL.Image.Image image mode=L size=1161x1392 at 0x13AE11760>, <PIL.Image.Image image mode=L size=1161x1392 at 0x13AE07B90>, <PIL.Image.Image image mode=L size=1161x1392 at 0x13AE13EF0>]


In [60]:
aquatint_head_ims_dark = [ImageChops.hard_light(Image.new('RGB',size=size),g_to_mono(im, (0,0,0))) for im in aquatint_head_ims]
zoom_aquatint_head_ims_dark = [ImageChops.hard_light(Image.new('RGB',size=size),g_to_mono(im, (0,0,0))) for im in zoom_aquatint_head_ims]
#aquatint_head_ims_dark = aquatint_head_ims
#zoom_aquatint_head_ims_dark = zoom_aquatint_head_ims

In [61]:
def flicker_marks_light(im, num_frames, bg_im, thresh_mult=1.7, alpha_min=0.5, add_alpha=False):
    thresh = threshold_otsu(np.asarray(im))
    im_grey = (im.copy()).convert("L")
    im_thresh = (np.asarray(im_grey) > thresh*thresh_mult)
    Strel = morph.disk(2)
    im_thresh = morph.dilation(im_thresh,Strel)

    label_img = measure.label(im_thresh)
    print("label_img.shape:", label_img.shape)
    #now we've got the labels, we just want to turn on and off
    #marks at random
    num_labels = label_img.max()
    print("num_labels:",num_labels)
    labels = np.arange(1, num_labels+1)
    frames_out = []
    for i in range(num_frames):
        alpha_vals = (255*np.random.uniform(low=alpha_min, high=1., size = len(labels))).astype(int)
        im_new = im_grey.copy()
        #Might be easiest to do this with alpha channel
        alpha = 10*np.ones(im_grey.size).astype(np.uint8).T
        for label,alpha_val in zip(labels, alpha_vals):
            alpha[np.where(label_img==label)] = alpha_val
        alpha_im = Image.fromarray(alpha)
        if add_alpha:
            im_new.putalpha(alpha_im)
        else:
            alpha_frac = alpha.astype(float)/255
            im_new_array = np.round( (1-alpha_frac)*bg_im.convert("L") + alpha_frac*np.array(im_grey.copy()) ).astype(np.uint8)
            im_new = Image.fromarray(im_new_array)
            
        frames_out.append(im_new)
        
    return frames_out, label_img


In [62]:
frames14_orig = []
fam_inds_14_orig = []

for i,aquatint_head_im_dark in enumerate(zoom_aquatint_head_ims_dark):
    flicker_light, _ = flicker_marks_light(aquatint_head_im_dark, 10, Image.new('RGB',size=size), thresh_mult=1.5)
    #turn green
    frames14_orig += [g_to_mono(f, (0,0,0)) for f in flicker_light]
    fam_inds_14_orig+=[i]*len(flicker_light)

frames14 = supplement_ims(supplement_ims(frames14_orig))
fam_inds_14 = supplement_fam_inds(supplement_fam_inds(fam_inds_14_orig))

frames14 = [ImageChops.add(f,d) for (f,d) in zip(frames14, dark_frames*10)]

label_img.shape: (1392, 1161)
num_labels: 425
label_img.shape: (1392, 1161)
num_labels: 690
label_img.shape: (1392, 1161)
num_labels: 1131
label_img.shape: (1392, 1161)
num_labels: 306
label_img.shape: (1392, 1161)
num_labels: 323
label_img.shape: (1392, 1161)
num_labels: 306
label_img.shape: (1392, 1161)
num_labels: 1131
label_img.shape: (1392, 1161)
num_labels: 690


In [63]:
v14 = VideoClip.DataVideoClip(frames14, im_to_frame_func(downsample), fps=fps)

#v13.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)

In [64]:
print(aquatint_head_ims_dark)

[<PIL.Image.Image image mode=RGB size=1161x1392 at 0x12FBFA570>, <PIL.Image.Image image mode=RGB size=1161x1392 at 0x12FF5E360>, <PIL.Image.Image image mode=RGB size=1161x1392 at 0x13AE04050>, <PIL.Image.Image image mode=RGB size=1161x1392 at 0x13AE2BE00>, <PIL.Image.Image image mode=RGB size=1161x1392 at 0x13AE105C0>, <PIL.Image.Image image mode=RGB size=1161x1392 at 0x13AE4EE70>, <PIL.Image.Image image mode=RGB size=1161x1392 at 0x13AE4EE40>, <PIL.Image.Image image mode=RGB size=1161x1392 at 0x13AE4EB40>, <PIL.Image.Image image mode=RGB size=1161x1392 at 0x13AE4EDE0>, <PIL.Image.Image image mode=RGB size=1161x1392 at 0x13AE4DA60>, <PIL.Image.Image image mode=RGB size=1161x1392 at 0x13AE4EE10>, <PIL.Image.Image image mode=RGB size=1161x1392 at 0x13AE4F020>]


In [ ]:
flicker_marks_light_dict_thresh1 = {}

frames15_orig = []
fam_inds_15_orig = []

for i,aquatint_head_im_dark in enumerate(aquatint_head_ims_dark*2):
    flicker_light, _ = flicker_marks_light(aquatint_head_im_dark, 6, Image.new('RGB',size=size), thresh_mult=1.5)
    frames15_orig += [g_to_mono(f, (0,0,0)) for f in flicker_light]
    fam_inds_15_orig+=[i]*len(flicker_light)

frames15 = supplement_ims(supplement_ims(frames15_orig))

frames15 = [ImageChops.add(f,d) for (f,d) in zip(frames15, dark_frames*10)]

alphas = np.linspace(0.,1.,len(frames15))
frames15 = [ImageChops.blend(f14,f15,alpha=alpha) for (f14,f15,alpha) in zip(frames14*10,frames15,alphas)]

label_img.shape: (1392, 1161)
num_labels: 818
label_img.shape: (1392, 1161)
num_labels: 965
label_img.shape: (1392, 1161)
num_labels: 1297


In [ ]:
#v15 = VideoClip.DataVideoClip(frames15, im_to_frame_func(8), fps=fps)
#v14.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)

In [ ]:
frames15 = frames15[:10]
frames_13to15 = frames13 + frames14 + frames15
frames_13to15 = add_text_dark(frames_13to15, text_ims[16:], advance_prob=0.8)
frames13, frames14, frames15 = (frames_13to15[:len(frames13)],
                                   frames_13to15[len(frames13) : len(frames13)+len(frames14)],
                                   frames_13to15[len(frames13)+len(frames14) :],
                                  )

v13,v14,v15 = (VideoClip.DataVideoClip(frames13, im_to_frame_func(downsample), fps=fps),
               VideoClip.DataVideoClip(frames14, im_to_frame_func(downsample), fps=fps),
               VideoClip.DataVideoClip(frames15, im_to_frame_func(downsample), fps=fps),
              )

In [ ]:
#v16 = VideoClip.DataVideoClip(frames16, im_to_frame_func(8), fps=fps)

v = concatenate_videoclips([v13,v14,v15])

v.write_videofile("test.mp4", codec='libx264', 
                     audio_codec='aac', 
                     temp_audiofile='temp-audio.m4a', 
                     remove_temp=False)

Now the same but a lower threshold 

In [ ]:
# frames17_orig = []
frames16_orig = []
fam_inds_16_orig = []


total_time = 3
head_ind = 0
reverse_prob = 0.1
t=0
while t < total_time:
    print("head_ind:",head_ind)
    num_frames = np.random.randint(4)
    flicker_light, _ = flicker_marks_light(aquatint_head_ims[head_ind], num_frames, 
                                           Image.new('RGB',size=size), thresh_mult=1.,
                                          alpha_min=0.5)
    #turn green
    flicker_marks_light_dict_thresh1[i] = flicker_light
    frames16_orig += [g_to_mono(f, (0,0,0)) for f in flicker_light]
    t += 1/fps * num_frames
    if (np.random.random()<reverse_prob) and (head_ind>0):
        head_ind -= 1
    elif head_ind<len(aquatint_head_ims)-1:
        head_ind += 1
    else:
        head_ind = 0

frames16 = supblend(supblend(supplement_ims(frames16_orig)))

frames16 = [ImageChops.add(f,d) for (f,d) in zip(frames16, dark_frames*10)]

In [ ]:

#v.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False, verbose=False, logger=None)

In [ ]:
v16 = VideoClip.DataVideoClip(frames16, im_to_frame_func(downsample), fps=fps)
print(v16.duration)
#v17.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)

In [ ]:
v7to16 = concatenate_videoclips([v7,v8,v9,v10,v11,v12,v13,v14,v15,v16])
audio7to16 = concatenate_audioclips([guitar_audio_clip2_wbackwards] * int(v7to16.duration/guitar_audio_clip2_wbackwards.duration+1) )
audio7to16_rain = concatenate_audioclips([rain_audio_clip] * int(v7to16.duration/rain_audio_clip.duration+1))
audio7to16 = CompositeAudioClip([audio7to16, audio7to16_rain])
audio7to16 = audio7to16.set_duration(v7to16.duration)
v7to16 = v7to16.set_audio(audio7to16)

v_upto_16 = concatenate_videoclips([v_1to6, v7to16])

#v_upto_16.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)
#

Maybe we return to the lightfield now. We could just a white bird...

In this first set, we invert the raven images, and use the value in 
the aquatint frames to set alpha of the raven 

In [ ]:
frames17_head = []

total_time = 5
reverse_prob = 0.2
t=0
while t < total_time:
    print("head_ind:",head_ind)
    num_frames = np.random.randint(5)
    flicker_light, _ = flicker_marks_light(aquatint_head_ims[head_ind], num_frames, 
                                           Image.new('RGB',size=size), thresh_mult=1.,
                                          alpha_min=0.5)
    #turn green
    flicker_marks_light_dict_thresh1[i] = flicker_light
    frames17_head += [g_to_mono(f, (0,0,0)) for f in flicker_light]
    t += 1/fps * num_frames
    if (np.random.random()<reverse_prob) and (head_ind>0):
        head_ind -= 1
    elif head_ind<len(aquatint_head_ims)-1:
        head_ind += 1
    else:
        head_ind = 0

frames17_head = supblend(supblend(supplement_ims(frames17_head)))

frames17_head = [ImageChops.add(f,d) for (f,d) in zip(frames17_head, dark_frames*10)]


raven_im_ind = 0

raven_im_ind = 0
frames_per_raven = 3 # number of frames per raven image

n_frames_this_raven = 0
num_raven_loops = 1
raven_loops_done = 0
frames17=[]
for i,f17 in enumerate(frames17_head):
    raven_im = ImageOps.invert(raven_ims_use[raven_im_ind].copy().convert("L"))
    #thresh = threshold_otsu(np.asarray(raven_im))
    #raven_im_inverted = ImageOps.invert(raven_im.copy())
    #im_thresh = (np.asarray(raven_im_inverted) > thresh)
    #grey_inds = np.where(im_thresh)
    
    #f8_grey_array = np.array(f8.convert("L").convert("RGB"))
    #frame_array[grey_inds] = f8_grey_array[grey_inds]
    #frame_array = np.array(f8.copy())
    #frame = Image.fromarray(frame_array)
    f17_grey = f17.convert("L")
    alpha = (np.array(f17_grey).astype(float)/255)

    raven_array = np.round( (1-alpha)*0 + (alpha)*np.array(raven_im) ).astype(np.uint8)
    raven_im = Image.fromarray(raven_array)
    raven_im = g_to_mono(raven_im, (0.,0.,0.))

    if n_frames_this_raven == frames_per_raven:
        if raven_im_ind<len(raven_ims_use)-1:
            raven_im_ind = raven_im_ind+1 
        else: 
            raven_im_ind = 0
            raven_loops_done += 1
            if raven_loops_done == num_raven_loops:
                break
        n_frames_this_raven = 0
    else:
        n_frames_this_raven += 1

    frames17.append(raven_im)

frames17 = [ImageChops.add(f17, f17_head) for (f17,f17_head) in zip(frames17,frames17_head)]


In [ ]:
v17 = VideoClip.DataVideoClip(frames17, im_to_frame_func(downsample), fps=fps)
v = concatenate_videoclips([eval("v%d"%i) for i in range(13,18)])
#v = concatenate_videoclips([v17,v18])
v.write_videofile("test.mp4", codec='libx264', 
                     audio_codec='aac', 
                     temp_audiofile='temp-audio.m4a', 
                     remove_temp=False)

In [ ]:
raven_frames = []

raven_im_ind = 0

raven_im_ind = 0
frames_per_raven = 3 # number of frames per raven image

n_frames_this_raven = 0
num_raven_loops = 1
raven_loops_done = 0

for i,f17 in enumerate(frames17):
    raven_im = ImageOps.invert(raven_ims_use[raven_im_ind].copy())
    #thresh = threshold_otsu(np.asarray(raven_im))
    #raven_im_inverted = ImageOps.invert(raven_im.copy())
    #im_thresh = (np.asarray(raven_im_inverted) > thresh)
    #grey_inds = np.where(im_thresh)

    if n_frames_this_raven == frames_per_raven:
        if raven_im_ind<len(raven_ims_use)-1:
            raven_im_ind = raven_im_ind+1 
        else: 
            raven_im_ind = 0
            raven_loops_done += 1
            if raven_loops_done == num_raven_loops:
                break
        n_frames_this_raven = 0
    else:
        n_frames_this_raven += 1

    raven_frames.append(g_to_mono(raven_im, (0,0,0)))

darken_facs = np.linspace(0.,1.,len(frames17))
frames18 = [ImageChops.add(darken_im(r,f), f17) for (r,f17,f) in zip(raven_frames,frames17,darken_facs)]
#frames18 = [ImageChops.blend(f17, r, a) for (r, f17, a) in zip(raven_frames,frames17,np.linspace(0.,1,len(frames17)))]

In [ ]:
v18 = VideoClip.DataVideoClip(frames18, im_to_frame_func(downsample), fps=fps)
#v19 = VideoClip.DataVideoClip(frames19, im_to_frame_func(8), fps=fps)

v = concatenate_videoclips([v16, v17,v18])

v.write_videofile("test.mp4", codec='libx264', 
                     audio_codec='aac', 
                     temp_audiofile='temp-audio.m4a', 
                     remove_temp=False)

In [ ]:
#Now zoom in on white raven
frames19 = []

raven_im_ind = 0

raven_im_ind = 0
frames_per_raven = 3 # number of frames per raven image

n_frames_this_raven = 0
num_raven_loops = 1
raven_loops_done = 0

for i,f16 in enumerate(frames16):
    raven_im_orig = ImageOps.invert(raven_ims_use_zoom[raven_im_ind].copy().convert("L"))
    #thresh = threshold_otsu(np.asarray(raven_im))
    #raven_im_inverted = ImageOps.invert(raven_im.copy())
    #im_thresh = (np.asarray(raven_im_inverted) > thresh)
    #grey_inds = np.where(im_thresh)
    
    #f8_grey_array = np.array(f8.convert("L").convert("RGB"))
    #frame_array[grey_inds] = f8_grey_array[grey_inds]
    #frame_array = np.array(f8.copy())
    #frame = Image.fromarray(frame_array)
    f16_grey = f16.convert("L")
    alpha = (np.array(f16_grey).astype(float)/255)**0.5

    raven_array = np.round( (1-alpha)*0 + (alpha)*np.array(raven_im_orig) ).astype(np.uint8)
    raven_im = Image.fromarray(raven_array)
    raven_im = g_to_mono(raven_im, (0.,0.,0.))

    if n_frames_this_raven == frames_per_raven:
        if raven_im_ind<len(raven_ims_use_zoom)-1:
            raven_im_ind = raven_im_ind+1 
        else: 
            raven_im_ind = 0
            raven_loops_done += 1
            if raven_loops_done == num_raven_loops:
                break
        n_frames_this_raven = 0
    else:
        n_frames_this_raven += 1

    frames19.append(
        ImageChops.add(
            ImageChops.add(f16, raven_im),
            g_to_mono(raven_im_orig, (0,0,0))
        )
    )
#frames19 = supblend(frames19)


In [ ]:
v19 = VideoClip.DataVideoClip(frames19, im_to_frame_func(downsample), fps=fps)
#v19 = VideoClip.DataVideoClip(frames19, im_to_frame_func(8), fps=fps)

#v19.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)

In [ ]:
#Now zoom in on white raven
frames20 = []

raven_im_ind = 0

raven_im_ind = 0
frames_per_raven = 3 # number of frames per raven image

n_frames_this_raven = 0
num_raven_loops = 1
raven_loops_done = 0

for i,f16 in enumerate(frames16):
    raven_im_orig = ImageOps.invert(raven_ims_use_zoomed[raven_im_ind].copy().convert("L"))
    #thresh = threshold_otsu(np.asarray(raven_im))
    #raven_im_inverted = ImageOps.invert(raven_im.copy())
    #im_thresh = (np.asarray(raven_im_inverted) > thresh)
    #grey_inds = np.where(im_thresh)
    
    #f8_grey_array = np.array(f8.convert("L").convert("RGB"))
    #frame_array[grey_inds] = f8_grey_array[grey_inds]
    #frame_array = np.array(f8.copy())
    #frame = Image.fromarray(frame_array)
    f16_grey = f16.convert("L")
    alpha = (np.array(f16_grey).astype(float)/255)**0.5

    raven_array = np.round( (1-alpha)*0 + (alpha)*np.array(raven_im_orig) ).astype(np.uint8)
    raven_im = Image.fromarray(raven_array)
    raven_im = g_to_mono(raven_im, (0.,0.,0.))

    if n_frames_this_raven == frames_per_raven:
        if raven_im_ind<len(raven_ims_use_zoomed)-1:
            raven_im_ind = raven_im_ind+1 
        else: 
            raven_im_ind = 0
            raven_loops_done += 1
            if raven_loops_done == num_raven_loops:
                break
        n_frames_this_raven = 0
    else:
        n_frames_this_raven += 1

    frames20.append(
        ImageChops.add(
            ImageChops.add(f16, raven_im),
            g_to_mono(raven_im_orig, (0,0,0))
        )
    )

#frames20 = supblend(frames20)

In [ ]:
v20 = VideoClip.DataVideoClip(frames20, im_to_frame_func(downsample), fps=fps)
#v19 = VideoClip.DataVideoClip(frames19, im_to_frame_func(8), fps=fps)

#v20.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)

In [ ]:
frames21 = [ImageChops.multiply(
    lighten_im(f20,lighten_fac), f1) for (f20,f1,lighten_fac) in zip(frames20, frames1, np.linspace(1.,0.,len(frames20)))]

In [ ]:
v21 = VideoClip.DataVideoClip(frames21, im_to_frame_func(downsample), fps=fps)
#v19 = VideoClip.DataVideoClip(frames19, im_to_frame_func(8), fps=fps)

#v21.write_videofile("test.mp4", codec='libx264', 
#                     audio_codec='aac', 
#                     temp_audiofile='temp-audio.m4a', 
#                     remove_temp=False)

In [ ]:
v7to21 = concatenate_videoclips([eval("v%d"%i) for i in range(7,22)])
#v7to15 = concatenate_videoclips([v7to12, v_13to15])
#v7to22 = concatenate_videoclips([v7to15] + eval("v%d"%i) for i in range(16,23)])

audio7to21 = concatenate_audioclips([guitar_audio_clip3_wbackwards] * int(v7to21.duration/guitar_audio_clip3_wbackwards.duration+1) )
audio7to21 = audio7to21.audio_fadeout(v21.duration + v20.duration)
audio7to21_rain = concatenate_audioclips([rain_audio_clip] * int(v7to21.duration/rain_audio_clip.duration+1))
audio7to21 = CompositeAudioClip([audio7to21, audio7to21_rain])
audio7to21 = audio7to21.set_duration(v7to21.duration)

v7to21 = v7to21.set_audio(audio7to21)

v = concatenate_videoclips([v_1to6,v7to21])

v.write_videofile("test.mp4", codec='libx264', 
                     audio_codec='aac', 
                     temp_audiofile='temp-audio.m4a', 
                     remove_temp=False, verbose=False)

In [ ]:
%matplotlib inline

display(frames18[12])

In [ ]:
frames2[0].save("carb_marks.jpg")

In [ ]:
frames7[23].save("soft_marks.jpg")

In [ ]:
frames18[12].save("aquatint_marks.jpg")